In [ ]:
import mne
import numpy as np
from mne.datasets import fetch_fsaverage
from mne.labels import labels_from_parc
import pandas as pd

# === SETUP ===
fs_dir = fetch_fsaverage(verbose=True)
subjects_dir = fs_dir
subject = 'fsaverage'

# Use MNE's built-in EASYCAP-M1 montage
montage = mne.channels.make_standard_montage('easycap-M1')

# === LOAD EEG DATA ===
raw = mne.io.read_raw_fif('your_raw.fif', preload=True)
raw.set_montage(montage)
raw.set_eeg_reference('average', projection=True)
raw.filter(1., 40.)

# === CREATE EVOKED ===
events = mne.make_fixed_length_events(raw, duration=15.0)  # 15s epochs
epochs = mne.Epochs(raw, events, tmin=0, tmax=15.0, baseline=None, preload=True)
evoked = epochs.average()

# === NOISE COVARIANCE ===
noise_cov = mne.compute_covariance(epochs, tmax=0., method='empirical')

# === SOURCE SPACE SETUP ===
src_surf = mne.setup_source_space(subject, spacing='ico5',
                                   subjects_dir=subjects_dir, add_dist=False)

src_vol = mne.setup_volume_source_space(subject,
    mri='aseg.mgz',
    pos=5.0,
    volume_label=['Brain-Stem', 'Left-Thalamus-Proper', 'Right-Thalamus-Proper'],
    bem=None,
    subjects_dir=subjects_dir,
    add_interpolator=True)

src = src_surf + src_vol

# === BEM SOLUTION ===
bem = mne.make_bem_solution(mne.make_bem_model(subject, subjects_dir=subjects_dir))

# === TRANS FILE ===
trans = 'fsaverage'

# === FORWARD + INVERSE ===
fwd = mne.make_forward_solution(evoked.info, trans=trans, src=src,
                                 bem=bem, eeg=True, meg=False)

inv_op = mne.minimum_norm.make_inverse_operator(evoked.info, fwd, noise_cov,
                                                loose=dict(surface=0.2, volume=1.0),
                                                depth=0.8)

stc_vec = mne.minimum_norm.apply_inverse(evoked, inv_op,
                                         lambda2=1./9., method='dSPM',
                                         pick_ori='vector')

# === ROI DEFINITIONS ===
cortical_roi_names = [
    'transversetemporal-lh', 'superiortemporal-lh', 'bankssts-lh',
    'middletemporal-lh', 'supramarginal-lh',
    'transversetemporal-rh', 'superiortemporal-rh', 'bankssts-rh',
    'middletemporal-rh', 'supramarginal-rh'
]

subcortical_roi_names = ['Brain-Stem', 'Left-Thalamus-Proper', 'Right-Thalamus-Proper']

labels_surf = mne.read_labels_from_annot(subject, parc='aparc', subjects_dir=subjects_dir)
roi_surf_labels = [lbl for lbl in labels_surf if lbl.name in cortical_roi_names]

labels_vol = labels_from_parc(src, parc='aseg', subjects_dir=subjects_dir)
roi_vol_labels = [lbl for lbl in labels_vol if lbl.name in subcortical_roi_names]

roi_labels = roi_surf_labels + roi_vol_labels

# === PCA-Based Time Series Extraction ===
def extract_roi_pca(stc_vec, labels, src):
    roi_pcs = []
    for label in labels:
        data = stc_vec.extract_label_time_course([label], src=src, mode='mean', return_generator=False)
        data = data[0]
        if data.ndim == 3:
            data = np.linalg.norm(data, axis=1)
        u, s, _ = np.linalg.svd(data.T, full_matrices=False)
        pc = u[:, 0] * s[0]
        roi_pcs.append(pc)
    return np.array(roi_pcs)

roi_timecourses = extract_roi_pca(stc_vec, roi_labels, src)

df = pd.DataFrame(roi_timecourses.T, columns=[lbl.name for lbl in roi_labels])
df['time'] = stc_vec.times

# Save result
df.to_csv('roi_timecourses_fsaverage.csv', index=False)


In [ ]:
import mne
from mne.datasets import fetch_fsaverage
import os
import numpy as np
from glob import glob


fs_dir = fetch_fsaverage(verbose=True)
subjects_dir = os.path.dirname(fs_dir)
subject = "fsaverage"
trans = "fsaverage"  # MNE has a built-in fsaverage transformation
src = os.path.join(fs_dir, "bem", "fsaverage-ico-5-src.fif")
bem = os.path.join(fs_dir, "bem", "fsaverage-5120-5120-5120-bem-sol.fif")
mri = os.path.join(fs_dir, "mri", "aseg.mgz")
montage = mne.channels.make_standard_montage("easycap-M1")

cortical_roi_names = [
    'transversetemporal-lh', 'superiortemporal-lh', 'bankssts-lh',
    'middletemporal-lh', 'supramarginal-lh',
    'transversetemporal-rh', 'superiortemporal-rh', 'bankssts-rh',
    'middletemporal-rh', 'supramarginal-rh'
]
subcortical_roi_names = ['Left-Thalamus-Proper', 'Right-Thalamus-Proper', 'Brain-Stem']

src_surf = mne.setup_source_space(subject, spacing="oct5", add_dist=False, subjects_dir=subjects_dir)

src_vol = mne.setup_volume_source_space(subject, mri=mri, pos=5.0, bem=bem, volume_label=subcortical_roi_names, subjects_dir=subjects_dir, add_interpolator=True, verbose=True)

src_mix = src_surf + src_vol



snr = 3.0  # use smaller SNR for raw data
inv_method = "dSPM"  # sLORETA, MNE, dSPM
parc = "aparc"  # the parcellation to use, e.g., 'aparc' 'aparc.a2009s'
loose_mix = dict(surface=0.2, volume=1.0)
depth = 0.8

labels_surf = mne.read_labels_from_annot(subject, parc='aparc', subjects_dir=subjects_dir)
roi_surf_labels = [lbl for lbl in labels_surf if lbl.name in cortical_roi_names]

#labels_vol = labels_from_parc(src, parc='aseg', subjects_dir=subjects_dir)
all_src_vertices = np.concatenate([s['vertno'] for s in src_mix])
vert_idx_map = {v: i for i, v in enumerate(all_src_vertices)}





In [ ]:
files_dir='//datadepot.rcac.purdue.edu/depot/heinz/data/UserTESTS/AM/EtardBrainstemAndComprehension/eeg/'
os.chdir(files_dir)
files=os.listdir()
for file in files:
    os.chdir(file)
    vhdrs=glob('*.vhdr')
    spont_seg=[]
    for eegs in vhdrs:
        raw = mne.io.read_raw_brainvision(eegs,preload=True)
        raw.set_montage(montage)
        raw.set_eeg_reference('average', projection=True)
        raw.filter(1., 40., fir_design='firwin')
        events, _ = mne.events_from_annotations(raw)
        if len(events) < 2:
            continue
        F = raw.info['sfreq']
        sp_t = (int(events[np.where(events[:, 2] == 1)[0],0])-0)/F
        spont = raw.crop(tmin=0, tmax=sp_t)
        spont_seg.append(spont)
    spont_comb = mne.concatenate_raws(spont_seg)
    eve_s = mne.make_fixed_length_events(spont_comb, start=0, stop=spont_comb.times[-1], duration=2.0)
    epochs = mne.Epochs(spont_comb, eve_s, tmin=0, tmax=2.0, baseline=None, preload=True)
    noise_cov = mne.compute_covariance(epochs, tmin=0.0, tmax=2.0, method='shrunk')
    
    str_save = file + 'noise_cov.fif'
    mne.write_cov(str_save, noise_cov, overwrite=True)


    fwd_mix = mne.make_forward_solution(epochs.info,trans, src_mix, bem, mindist=5.0, meg=False, eeg=True, n_jobs=None)
    inverse_operator_mix = mne.minimum_norm.make_inverse_operator(epochs.info, fwd_mix, noise_cov, depth=depth, loose=loose_mix, verbose=True)

    for eegs in vhdrs:
        raw = mne.io.read_raw_brainvision(eegs,preload=True)
        raw.set_montage(montage)
        raw.set_eeg_reference('average', projection=True)
        raw.filter(1., 40., fir_design='firwin')
        events, event_id = mne.events_from_annotations(raw)
        F = raw.info['sfreq']
        seg_t = (events[-1][0]-events[-2][0])/F
        epoch = mne.Epochs(raw, events, event_id = events[-2][2], tmin=0.0, tmax=seg_t, baseline=None, preload=True)
        evoked = epoch.average()
        stc_vec = mne.minimum_norm.apply_inverse(evoked, inverse_operator_mix, lambda2=1./9., method='dSPM', pick_ori='vector')

        roi_pcs = {}
        for label in roi_surf_labels:
            print(label)
            data = stc_vec.extract_label_time_course([label], src=src_mix, mode="mean")
            data = data[0]
            if data.ndim == 3:
                data = np.linalg.norm(data, axis=1)  # collapse x/y/z to magnitude
            u, s, _ = np.linalg.svd(data.T, full_matrices=False)
            pc = u[:, 0] * s[0]  # first principal component
            roi_pcs[label.name]=pc

        for label in subcortical_roi_names:
            print(label)
            vol_src = [s for s in src_mix if s['type'] == 'vol' and s.get('seg_name') == label]
            vertno = vol_src[0]['vertno']
            selected_indices = [vert_idx_map[v] for v in vertno if v in vert_idx_map]
            data = stc_vec.data[selected_indices, :]
            if data.ndim == 3:
                data = np.linalg.norm(data, axis=1)
            u, s, _ = np.linalg.svd(data.T, full_matrices=False)
            pc = u[:, 0] * s[0]  # first principal component
            roi_pcs[label]=pc
        
        str_save = eegs[:-5] + '.npz'

        np.savez_compressed(str_save, **roi_pcs)

    os.chdir    

    os.chdir('../')       

In [ ]:
files_dir='//datadepot.rcac.purdue.edu/depot/heinz/data/UserTESTS/AM/EtardBrainstemAndComprehension/eeg/YH00/'
os.chdir(files_dir)
vhdrs=glob('*.vhdr')
spont_seg=[]
for eegs in vhdrs:
    raw = mne.io.read_raw_brainvision(eegs,preload=True)
    raw.set_montage(montage)
    raw.set_eeg_reference('average', projection=True)
    raw.filter(1., 40., fir_design='firwin')
    events, _ = mne.events_from_annotations(raw)
    if len(events) < 2:
        continue
    F = raw.info['sfreq']
    sp_t = (events[1][0]-events[0][0])/F
    spont = raw.crop(tmin=0, tmax=sp_t)
    spont_seg.append(spont)
spont_comb = mne.concatenate_raws(spont_seg)
eve_s = mne.make_fixed_length_events(spont_comb, start=0, stop=spont_comb.times[-1], duration=2.0)
epochs = mne.Epochs(spont_comb, eve_s, tmin=0, tmax=2.0, baseline=None, preload=True)
noise_cov = mne.compute_covariance(epochs, tmin=0.0, tmax=2.0, method='shrunk')

fwd_mix = mne.make_forward_solution(epochs.info,trans, src_mix, bem, mindist=5.0, meg=False, eeg=True, n_jobs=None)
inverse_operator_mix = mne.minimum_norm.make_inverse_operator(epochs.info, fwd_mix, noise_cov, depth=depth, loose=loose_mix, verbose=True)

In [ ]:
raw = mne.io.read_raw_brainvision(vhdrs[0],preload=True)
raw.set_montage(montage)
raw.set_eeg_reference('average', projection=True)
raw.filter(1., 40.)
events, event_id = mne.events_from_annotations(raw)
F = raw.info['sfreq']
seg_t = (events[-1][0]-events[-2][0])/F
epoch = mne.Epochs(raw, events, event_id = events[-2][2], tmin=0.0, tmax=seg_t, baseline=None, preload=True)
evoked = epoch.average()
stc_vec = mne.minimum_norm.apply_inverse(evoked, inverse_operator_mix, lambda2=1./9., method='dSPM', pick_ori='vector')

In [ ]:
labels_surf = mne.read_labels_from_annot(subject, parc='aparc', subjects_dir=subjects_dir)
roi_surf_labels = [lbl for lbl in labels_surf if lbl.name in cortical_roi_names]
roi_pcs = {}
for label in roi_surf_labels:
    print(label)
    data = stc_vec.extract_label_time_course([label], src=src_mix, mode="mean")
    data = data[0]
    if data.ndim == 3:
        data = np.linalg.norm(data, axis=1)  # collapse x/y/z to magnitude
    u, s, _ = np.linalg.svd(data.T, full_matrices=False)
    pc = u[:, 0] * s[0]  # first principal component
    roi_pcs[label.name]=pc


all_src_vertices = np.concatenate([s['vertno'] for s in src_mix])
vert_idx_map = {v: i for i, v in enumerate(all_src_vertices)}
for label in subcortical_roi_names:
    print(label)
    vol_src = [s for s in src_mix if s['type'] == 'vol' and s.get('seg_name') == label]
    vertno = vol_src[0]['vertno']
    selected_indices = [vert_idx_map[v] for v in vertno if v in vert_idx_map]
    data = stc_vec.data[selected_indices, :]
    if data.ndim == 3:
        data = np.linalg.norm(data, axis=1)
    u, s, _ = np.linalg.svd(data.T, full_matrices=False)
    pc = u[:, 0] * s[0]  # first principal component
    roi_pcs[label]=pc


    


In [ ]:
print(src_mix[2].get('seg_name'))
print(src_mix[2]['type'])
vol_src = [s for s in src_mix if s['type'] == 'vol' and s.get('seg_name') == 'Left-Thalamus-Proper']
thalamus_vertno=vol_src[0]['vertno']
all_src_vertices = np.concatenate([s['vertno'] for s in src_mix])
vert_idx_map = {v: i for i, v in enumerate(all_src_vertices)}
selected_indices = [vert_idx_map[v] for v in thalamus_vertno if v in vert_idx_map]
len(selected_indices) == 0
tc_thalamus_lh = stc_vec.data[selected_indices, :]
tc_thalamus_lh.ndim

In [ ]:
labels_surf = mne.read_labels_from_annot(subject, parc='aparc', subjects_dir=subjects_dir)
roi_surf_labels = [lbl for lbl in labels_surf if lbl.name in cortical_roi_names]
roi_pcs = []
for label in roi_surf_labels:
    data = stc_vec.extract_label_time_course([label], src=src_mix, mode="mean")
    data = data[0]
    if data.ndim == 3:
        data = np.linalg.norm(data, axis=1)  # collapse x/y/z to magnitude
    u, s, _ = np.linalg.svd(data.T, full_matrices=False)
    pc = u[:, 0] * s[0]  # first principal component
    roi_pcs.append(pc)



In [ ]:
raw = mne.io.read_raw_brainvision(vhdrs[0],preload=True)
raw.set_montage(montage)
raw.set_eeg_reference('average', projection=True)
raw.filter(1., 40.)
events, event_id = mne.events_from_annotations(raw)
F = raw.info['sfreq']
seg_t = (events[-1][0]-events[-2][0])/F
epoch = mne.Epochs(raw, events, event_id = events[-2][2], tmin=0.0, tmax=seg_t, baseline=None, preload=True)
evoked = epochs.average()
stc_vec = mne.minimum_norm.apply_inverse(evoked, inverse_operator_mix, lambda2=1./9., method='dSPM', pick_ori='vector')
roi_pcs = []
for label in cortical_roi_names:
    data = stc_vec.extract_label_time_course([label], src=src_mix, mode="mean_flip")
    data = data[0]
    if data.ndim == 3:
        data = np.linalg.norm(data, axis=1)  # collapse x/y/z to magnitude
    u, s, _ = np.linalg.svd(data.T, full_matrices=False)
    pc = u[:, 0] * s[0]  # first principal component
    roi_pcs.append(pc)
    
#roi_timecourses = extract_roi_pca(stc_vec, roi_labels, src_mix)



In [ ]:
import os
import numpy as np
from glob import glob
import mne
files_dir='//datadepot.rcac.purdue.edu/depot/heinz/data/UserTESTS/AM/EtardBrainstemAndComprehension/eeg/'
os.chdir(files_dir)
files=os.listdir()
src_tc_nm=[]
for file in files:
    os.chdir(file)
    vhdrs=glob('*.npz')
    for vhdr in vhdrs:
        src_tc_nm.append(file+'/'+vhdr)
    os.chdir('../')

batch_size = 10
for ii in range(len(src_tc_nm) - batch_size +1): # multiple batches
    iters = src_tc_nm[ii:ii+batch_size]
    yy=[]
    xx=[]
    for it in iters:
        data = np.load(it)
        dict = {k: data[k] for k in data.files}
        yy.append(np.stack([data[key] for key in data.files]))

        raw = mne.io.read_raw_brainvision(it[:-4]+'.vhdr',preload=True)
        raw.filter(1., 40.)
        events, event_id = mne.events_from_annotations(raw)
        F = raw.info['sfreq']
        seg_t = (events[-1][0]-events[-2][0])/F
        epoch = mne.Epochs(raw, events, event_id = events[-2][2], tmin=0.0, tmax=seg_t, baseline=None, preload=True)
        evoked = epoch.average()
        xx.append(evoked.get_data())

        ## ML code follows with input xx adn output yy

In [ ]:
len(src_tc_nm)
batch_size = 10
for ii in range(len(src_tc_nm) - batch_size +1): # multiple batches
    iters = src_tc_nm[ii:ii+batch_size]
    yy=[]
    xx=[]
    for it in iters:
        data = np.load(it)
        




In [ ]:
ii=0
batch_size = 10
iters = src_tc_nm[ii:ii+batch_size]
yy=[]
xx=[]
for it in iters:
    data = np.load(it)
    dict = {k: data[k] for k in data.files}
    yy.append(np.stack([data[key] for key in data.files]))

    raw = mne.io.read_raw_brainvision(it[:-4]+'.vhdr',preload=True)
    raw.filter(1., 40.)
    events, event_id = mne.events_from_annotations(raw)
    F = raw.info['sfreq']
    seg_t = (events[-1][0]-events[-2][0])/F
    epoch = mne.Epochs(raw, events, event_id = events[-2][2], tmin=0.0, tmax=seg_t, baseline=None, preload=True)
    evoked = epoch.average()
    xx.append(evoked.get_data())

    ## ML code follows
    

    


In [ ]:
yy

In [ ]:
import mne
raw = mne.io.read_raw_brainvision(it[:-4]+'.vhdr',preload=True)
raw.filter(1., 40.)
events, event_id = mne.events_from_annotations(raw)
F = raw.info['sfreq']
seg_t = (events[-1][0]-events[-2][0])/F
epoch = mne.Epochs(raw, events, event_id = events[-2][2], tmin=0.0, tmax=seg_t, baseline=None, preload=True)
evoked = epoch.average()

In [ ]:
dt=evoked.get_data()

In [ ]:
yy[-1]

In [ ]:
dt